1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  

In [3]:
import requests

In [5]:
response = requests.get('https://realpython.github.io/fake-jobs/')

In [7]:
from bs4 import BeautifulSoup as bs

In [9]:
soup = bs(response.text)

In [11]:
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Fake Python</title>
<link href="https://cdn.jsdelivr.net/npm/bulma@0.9.2/css/bulma.min.css" rel="stylesheet"/>
</head>
<body>
<section class="section">
<div class="container mb-5">
<h1 class="title is-1">
        Fake Python
      </h1>
<p class="subtitle is-3">
        Fake Jobs for Your Web Scraping Journey
      </p>
</div>
<div class="container">
<div class="columns is-multiline" id="ResultsContainer">
<div class="column is-half">
<div class="card">
<div class="card-content">
<div class="media">
<div class="media-left">
<figure class="image is-48x48">
<img alt="Real Python Logo" src="https://files.realpython.com/media/real-python-logo-thumbnail.7f0db70c2ed2.jpg?__no_cf_polish=1"/>
</figure>
</div>
<div class="media-content">
<h2 class="title is-5">Senior Python Developer</h2>
<h3 class="subtitle is-6 company">Payne, Roberts and Davis</h3>
</div>
</div>


a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  

In [13]:
s_p_d = soup.find('h2', class_='title is-5')

In [15]:
s_p_d

<h2 class="title is-5">Senior Python Developer</h2>

b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.

In [17]:
jobs = soup.findAll('h2')
companies = soup.findAll('h3')
locations = soup.findAll('p', class_ = 'location')
posting_dates =  soup.findAll('time')

In [19]:
x = 0
all_jobs = []
for job in jobs:
    job_dict = {}
    job_dict['title'] = jobs[x].text
    job_dict['company'] = companies[x].text
    job_dict['location'] = locations[x].text.replace('\n','').strip()
    job_dict['posted_date'] = posting_dates[x].text
    all_jobs.append(job_dict)
    x += 1

c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.

d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [21]:
import pandas as pd

In [23]:
jobs_df = pd.DataFrame(all_jobs)

In [25]:
jobs_df

,title,company,location,posted_date
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.   

In [ ]:
jobs_df['apply_url'] = None

In [ ]:
jobs_df

a. First, use the BeautifulSoup find_all method to extract the urls.

In [ ]:
apply_urls = soup.findAll('a')

In [ ]:
x = 0
for url in apply_urls:
    if url.text != 'Apply':
        del apply_urls[x] 
    x += 1

In [ ]:
apply_urls

    b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [ ]:
base_apply_url = 'https://realpython.github.io/fake-jobs/jobs/'
job_url_number = 0
apply_urls_b = soup.findAll('a')
apply_urls_list = []
for job, desc in jobs_df.iterrows():
    job_url = desc['title'].lower().replace(' ','-').replace('(','').replace(')','').replace(',','').replace('/','-') + '-' + str(job_url_number) + '.html'
    job_url =  base_apply_url + job_url
    job_url_number += 1
    jobs_df.at[job, 'apply_url'] = job_url
    apply_urls_list.append(job_url)

3. Finally, we want to get the job description text for each job.  

    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  

In [ ]:
r = requests.get(apply_urls_list[0])

In [ ]:
soup = bs(r.text)

In [ ]:
description = soup.findAll('p')[1].text

In [ ]:
description

    b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  

In [ ]:
def desc_fetcher(url):
    r = requests.get(url)
    soup = bs(r.text)
    description = soup.findAll('p')[1].text
    return description

In [ ]:
desc_fetcher('https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html')

    c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [ ]:
jobs_df['job_description'] = jobs_df['apply_url'].apply(desc_fetcher)

In [ ]:
jobs_df

BONUS

1. Navigate to https://www.billboard.com/charts/hot-100/. Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.

In [47]:
r = requests.get('https://www.billboard.com/charts/hot-100/')

In [49]:
soup = bs(r.text)

In [51]:
top_100 = soup.findAll('div', class_ = 'o-chart-results-list-row-container')

In [143]:
all_songs = []
for song_row in top_100:
    song = {}
    rows = song_row.findAll('li', class_='o-chart-results-list__item')
    song['rank'] = rows[0].text.replace('\n','').replace('NEW','').replace('\t','').replace('RE-ENTRY','').strip()
    song['title'] = rows[3].text.replace('\n','').split('\t')[9]
    song['artist'] = rows[3].text.replace('\n','').split('\t')[-1]
    song['last_week'] = rows[6].text.replace('\n','').replace('-','None').strip()
    song['peak_pos'] = rows[7].text.replace('\n','').strip()
    song['weeks_on_chart'] = rows[8].text.replace('\n','').strip()
    all_songs.append(song)

In [145]:
top_100_df = pd.DataFrame(all_songs)

In [147]:
top_100_df

,rank,title,artist,last_week,peak_pos,weeks_on_chart
0,1,Luther,Kendrick Lamar & SZA,1,1,16
1,2,Die With A Smile,Lady Gaga & Bruno Mars,2,1,30
2,3,Not Like Us,Kendrick Lamar,3,1,44
3,4,A Bar Song (Tipsy),Shaboozey,5,1,48
4,5,TV Off,Kendrick Lamar Featuring Lefty Gunplay,4,2,16
...,...,...,...,...,...,...
95,96,Voy A Llevarte Pa PR,Bad Bunny,86,17,10
96,97,Texas,Blake Shelton,None,97,1
97,98,Tu Boda,Oscar Maydon & Fuerza Regida,94,22,20
98,99,ExtraL,JENNIE & Doechii,None,75,2


2. After getting the code working for the current chart, navigate to last week's chart. Notice how the url for the page changes. Write a function which will, given a date, return a pandas DataFrame containing the Billboard chart data for that date.

In [216]:
def chart_fetcher(date):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    target_url = 'https://www.billboard.com/charts/hot-100/' + date + '/'
    print(target_url)
    r = requests.get(target_url, headers = headers)
    print(r)
    soup = bs(r.text)
    top_100 = soup.findAll('div', class_ = 'o-chart-results-list-row-container')
    all_songs = []
    for song_row in top_100:
        song = {}
        rows = song_row.findAll('li', class_='o-chart-results-list__item')
        song['rank'] = rows[0].text.replace('\n','').replace('NEW','').replace('\t','').replace('RE-ENTRY','').strip()
        song['title'] = rows[3].text.replace('\n','').split('\t')[9]
        song['artist'] = rows[3].text.replace('\n','').split('\t')[-1]
        song['last_week'] = rows[6].text.replace('\n','').replace('-','None').strip()
        song['peak_pos'] = rows[7].text.replace('\n','').strip()
        song['weeks_on_chart'] = rows[8].text.replace('\n','').strip()
        all_songs.append(song)
    top_100_df = pd.DataFrame(all_songs)
    return top_100_df

3. Write a loop to retrieve the Billboard chart data for the last 10 weeks.

In [200]:
chart_dfs = []
from datetime import datetime, timedelta
today = datetime.today()
dates = []
for i in range(10):
    week_ago = today - timedelta(weeks=i)
    dates.append(week_ago.strftime('%Y-%m-%d'))

In [218]:
for date in dates:
    print(date)
    print(chart_fetcher(date))

2025-03-24
https://www.billboard.com/charts/hot-100/2025-03-24/
<Response [200]>
Empty DataFrame
Columns: []
Index: []
2025-03-17
https://www.billboard.com/charts/hot-100/2025-03-17/
<Response [200]>
   rank                 title                                  artist  \
0     1                Luther                    Kendrick Lamar & SZA   
1     2      Die With A Smile                  Lady Gaga & Bruno Mars   
2     3           Not Like Us                          Kendrick Lamar   
3     4    A Bar Song (Tipsy)                               Shaboozey   
4     5                TV Off  Kendrick Lamar Featuring Lefty Gunplay   
..  ...                   ...                                     ...   
95   96  Voy A Llevarte Pa PR                               Bad Bunny   
96   97                 Texas                           Blake Shelton   
97   98               Tu Boda            Oscar Maydon & Fuerza Regida   
98   99                ExtraL                        JENNIE & Doechii 

BONUS NIGHTMARE

1. Start by using either the inspector or by viewing the page source. Can you identify a tag that might be helpful for finding the names of all performers? For now, just worry about the headliner and don't worry about the opener. (Eg. For Vince Gill, featuring Wendy Moten, we only care about Vince Gill.) Make use of this to create a list containing just the names of each inductee.

In [222]:
r = requests.get('https://www.ryman.com/events/')

In [224]:
soup = bs(r.text)

In [236]:
events = soup.findAll('div',class_ = 'info clearfix')

In [306]:
headliner_list = []
event_time_list = []
for event in events:
    cards = event.findAll('a', title='More Info')
    info_list = []
    for info in cards:
        parsed = info.text.replace('\n','').split('title="More Info">')[0]
        info_list.append(parsed)
    event_datetime = info_list[0]
    headliner = info_list[1]
    event_time_list.append(event_datetime)
    headliner_list.append(headliner)

In [308]:
headliner_list

['',
 'Band As One Nashville with Trisha Yearwood & Friends',
 'Joy Oladokun',
 'Wilder Woods',
 'The Righteous Brothers',
 'What The World Needs Now: The Bacharach Songbook Live',
 'The Road Taping',
 'Mark Normand',
 'Ryman Sidewalk Sessions with Kiran Gupta',
 "Umphrey's McGee presents umBOWL XI",
 'HANDSOME: LIVE with Tig Notaro, Fortune Feimster & Mae Martin',
 "Kid Rock's Comedy Jam",
 'Nateland Presents']


2. Next, try and find a tag that could be used to find the date and time for each show. Extract these into a list. Challenge: Convert these into two lists, one containing the date and the other containing the time. (Eg. split Mar 9, 2023 8:00 PM into Mar 9, 2023 and 8:00 PM.) 

In [391]:
date_list = []
time_list = []
for event_time in event_time_list:
    event_time = event_time.strip()
    if len(event_time) == 20:
        event_time = event_time.split(' ')
        if event_time[1] == '':
            del event_time[1]
        month = event_time[0]
        day = event_time[1]
        year = event_time[2]
        time = event_time[3]
        am_pm = event_time[4]
        date = [month,day,year]
        time = [time,am_pm]
        date_list.append(date)
        time_list.append(time)
    if len(event_time) == 15:
        event_time = event_time.replace('- ','-').split(' ')
        month = event_time[0]
        days = event_time[2]
        year = event_time[3]
        date = [month,days,year]
        time = [None,None]
        date_list.append(date)
        time_list.append(time)

3. Take the lists you created on parts 1 and 2 and convert them into a pandas DataFrame.

In [393]:
date_list,time_list,headliner_list

([['Mar', '26,', '2025'],
  ['Mar', '27,', '2025'],
  ['Mar', '29,', '2025'],
  ['Mar', '30,', '2025'],
  ['Mar', '31,', '2025'],
  ['Apr', '2,', '2025'],
  ['Apr', '3,', '2025'],
  ['Apr', '4,', '2025'],
  ['Apr', '4-5,', '2025'],
  ['Apr', '6,', '2025'],
  ['Apr', '7,', '2025'],
  ['Apr', '8,', '2025']],
 [['7:00', 'PM'],
  ['7:30', 'PM'],
  ['8:00', 'PM'],
  ['7:30', 'PM'],
  ['8:00', 'PM'],
  ['7:00', 'PM'],
  ['7:00', 'PM'],
  ['5:30', 'PM'],
  [None, None],
  ['7:00', 'PM'],
  ['7:00', 'PM'],
  ['7:00', 'PM']],
 ['',
  'Band As One Nashville with Trisha Yearwood & Friends',
  'Joy Oladokun',
  'Wilder Woods',
  'The Righteous Brothers',
  'What The World Needs Now: The Bacharach Songbook Live',
  'The Road Taping',
  'Mark Normand',
  'Ryman Sidewalk Sessions with Kiran Gupta',
  "Umphrey's McGee presents umBOWL XI",
  'HANDSOME: LIVE with Tig Notaro, Fortune Feimster & Mae Martin',
  "Kid Rock's Comedy Jam",
  'Nateland Presents'])

In [397]:
del headliner_list[0]
event_dict = {'headliner': headliner_list, 'date': date_list, 'time': time_list}
event_df = pd.DataFrame(event_dict)

In [399]:
event_df

,headliner,date,time
0,Band As One Nashville with Trisha Yearwood & F...,"[Mar, 26,, 2025]","[7:00, PM]"
1,Joy Oladokun,"[Mar, 27,, 2025]","[7:30, PM]"
2,Wilder Woods,"[Mar, 29,, 2025]","[8:00, PM]"
3,The Righteous Brothers,"[Mar, 30,, 2025]","[7:30, PM]"
4,What The World Needs Now: The Bacharach Songbo...,"[Mar, 31,, 2025]","[8:00, PM]"
5,The Road Taping,"[Apr, 2,, 2025]","[7:00, PM]"
6,Mark Normand,"[Apr, 3,, 2025]","[7:00, PM]"
7,Ryman Sidewalk Sessions with Kiran Gupta,"[Apr, 4,, 2025]","[5:30, PM]"
8,Umphrey's McGee presents umBOWL XI,"[Apr, 4-5,, 2025]","[None, None]"
9,"HANDSOME: LIVE with Tig Notaro, Fortune Feimst...","[Apr, 6,, 2025]","[7:00, PM]"


4. **Bonus #1:**: Add to your data frame the opening act for all shows that list an opener.

In [402]:
event_df['opening_act'] = None

In [404]:
event_df

,headliner,date,time,opening_act
0,Band As One Nashville with Trisha Yearwood & F...,"[Mar, 26,, 2025]","[7:00, PM]",None
1,Joy Oladokun,"[Mar, 27,, 2025]","[7:30, PM]",None
2,Wilder Woods,"[Mar, 29,, 2025]","[8:00, PM]",None
3,The Righteous Brothers,"[Mar, 30,, 2025]","[7:30, PM]",None
4,What The World Needs Now: The Bacharach Songbo...,"[Mar, 31,, 2025]","[8:00, PM]",None
5,The Road Taping,"[Apr, 2,, 2025]","[7:00, PM]",None
6,Mark Normand,"[Apr, 3,, 2025]","[7:00, PM]",None
7,Ryman Sidewalk Sessions with Kiran Gupta,"[Apr, 4,, 2025]","[5:30, PM]",None
8,Umphrey's McGee presents umBOWL XI,"[Apr, 4-5,, 2025]","[None, None]",None
9,"HANDSOME: LIVE with Tig Notaro, Fortune Feimst...","[Apr, 6,, 2025]","[7:00, PM]",None


In [434]:
opening_acts = soup.findAll('div',class_ = 'info clearfix')

In [514]:
for opening_act in opening_acts:
    if opening_act == opening_acts[0]:
        continue
    headliner = opening_act.find_all('a',title='More Info')[1].text.strip()
    if opening_act.find_all('h4'):
        opener = opening_act.find_all('h4')[0].text.strip()
    if not opening_act.find_all('h4'):
        opener = None
    print(opener)
    event_df.loc[event_df['headliner'] == headliner, 'opening_act'] = opener
    print('------------------------------------------------------------------')

with special guests Lauren Alaina, Kristin Chenoweth, Terri Clark, Anita Cochran, Sheryl Crow, Amy Grant, Keith Urban, and Lainey Wilson
------------------------------------------------------------------
with Medium Build
------------------------------------------------------------------
with HAFFWAY
------------------------------------------------------------------
None
------------------------------------------------------------------
featuring Todd Rundgren and Rob Shirakbari
------------------------------------------------------------------
None
------------------------------------------------------------------
None
------------------------------------------------------------------
Free & Open to the Public
------------------------------------------------------------------
None
------------------------------------------------------------------
None
------------------------------------------------------------------
None
---------------------------------------------------------------

In [516]:
event_df

,headliner,date,time,opening_act
0,Band As One Nashville with Trisha Yearwood & F...,"[Mar, 26,, 2025]","[7:00, PM]","with special guests Lauren Alaina, Kristin Che..."
1,Joy Oladokun,"[Mar, 27,, 2025]","[7:30, PM]",with Medium Build
2,Wilder Woods,"[Mar, 29,, 2025]","[8:00, PM]",with HAFFWAY
3,The Righteous Brothers,"[Mar, 30,, 2025]","[7:30, PM]",None
4,What The World Needs Now: The Bacharach Songbo...,"[Mar, 31,, 2025]","[8:00, PM]",featuring Todd Rundgren and Rob Shirakbari
5,The Road Taping,"[Apr, 2,, 2025]","[7:00, PM]",None
6,Mark Normand,"[Apr, 3,, 2025]","[7:00, PM]",None
7,Ryman Sidewalk Sessions with Kiran Gupta,"[Apr, 4,, 2025]","[5:30, PM]",Free & Open to the Public
8,Umphrey's McGee presents umBOWL XI,"[Apr, 4-5,, 2025]","[None, None]",None
9,"HANDSOME: LIVE with Tig Notaro, Fortune Feimst...","[Apr, 6,, 2025]","[7:00, PM]",None


5. **Bonus #2:**: Now, let's see if we can get the results beyond the first page. For this, you'll need to Web Developer Tools of your browser and navigate to the Network tab. Click the "Load More Events" button and you should see a GET request to the www.ryman.com domain.  
    a. Inspect this request and you should see that it goes to a URL like "https://www.ryman.com/events/events_ajax/24?category=0&venue=0&team=0&exclude=&per_page=12&came_from_page=event-list-page". In your Jupyter notebook, send a get request to this url and inspect the results.  
    b. You should find that the results that you get are HTML, but that they are not exactly formatted in a way that can be parsed. See if you can clean up the results set so that you can extract out the same information as above.  
    c. Create a DataFrame that contains data for the next 60 shows.

In [582]:
r = requests.get("https://www.ryman.com/events/events_ajax/24?category=0&venue=0&team=0&exclude=&per_page=60&came_from_page=event-list-page")

In [583]:
r = r.text.replace('\n',' ')

In [586]:
r = r.replace('\\','')

In [588]:
r = r.split('</div>t</div>')

In [664]:
x = 0
for item in r:
    print(x)
    x += 1
    try:
        item = item.split('title=')
        url = item[0].split('href=')[1].split('id=')[0].replace('\"','')
        title = item[1].split('tabindex=')[0].replace('More Info for ','').replace('\"','')
        date = item[2].split('</span>')
        month = date[0].split('class="m-date__month">')[1].strip()
        day_1 = date[1].replace('<span class="m-date__day">','').strip()
        day_2 = None
        year = date[2].replace('<span class="m-date__year">, ','')
        if date[2] == '<span class="m-date__separator">-':
            day_2 = date[3].replace('<span class="m-date__day">','').strip()
            year = date[4].replace('<span class="m-date__year">, ','')
        print(url)
        print(title)
        print(month)
        print(day_1,day_2)
        print(year)
        print('=====================================================================================')
    except:
        for data in item:
            print(data)
        print('error!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

0
https://www.ryman.com/event/2025-04-25-stavros-halkias-at-7-pm 
Stavros Halkias 
Apr
25 None
2025
1
https://www.ryman.com/event/sidewalk-sessions-cameron-reese 
Ryman Sidewalk Sessions with Cameron Reese 
Apr
26 None
2025
2
https://www.ryman.com/event/2025-04-26-rodney-atkins-at-8-pm 
Rodney Atkins 
Apr
26 None
2025
3
https://www.ryman.com/event/2025-04-28-japanese-breakfast-at-7-30-pm 
Japanese Breakfast 
Apr
28 None
2025
4
https://www.ryman.com/event/2025-lucy-dacus 
Lucy Dacus 
Apr
29 30
2025
5
https://www.ryman.com/event/sidewalk-sessions-tatum-scott 
Ryman Sidewalk Sessions with Tatum Scott 
May
1 None
2025
6
https://www.ryman.com/event/2025-05-01-maddie-tae-at-7-30-pm 
Maddie &amp; Tae 
May
1 None
2025
7
https://www.ryman.com/event/2025-drew-ellie-holcomb 
Drew &amp; Ellie Holcomb 
May
2 3
2025
8
https://www.ryman.com/event/2025-05-06-lea-michele-live-at-7-30-pm 
Lea Michele Live 
May
6 None
2025
9
https://www.ryman.com/event/sidewalk-sessions-isabel-pless 
Ryman Sidewalk Sessi